In [1]:
import pandas as pd
import sqlalchemy
import pymysql
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
print('sqlachemy version: ',sqlalchemy.__version__)
from time import time
from pathlib import Path
import os
from dotenv import load_dotenv
#load_dotenv()
from datetime import datetime

sqlachemy version:  1.4.39


In [2]:
def create_in_mysql(user, password, host, database):

    # Creating Postgre SQL connection, passing user and password as arguments when running script
    con_string=f'mysql+pymysql://{user}:{password}@{host}/{database}'
    engine = create_engine(con_string)

    # Tables to create from each csv file
    tables = ['dim_customers','dim_products','fact_sales','state_region_mapping']

    # Go thru each of the csv files to create tables and populate in Postgres
    for table in tables:

        t_start = time()

        # Creating an iterator
        df_iter = pd.read_csv(f'.\Datasets\{table}.csv', iterator=True, chunksize=1000, encoding = 'latin-1')
        #df = next(df_iter)

        # Getting only the columns (n=0) to create table and headers
        if 'dim' in table:
            table_name = table.replace('dim','tbl')
        else:
            pre_str = 'tbl_'
            table_name = pre_str + table
            
        
        #df.head(0).to_sql(name=table_name, con=engine, if_exists='replace', index=False)
        #df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

        # While there is data, insert next chunk
        while True: 

            try:
                df = next(df_iter)
                df.columns = [col.lower().replace(' ','_') for col in df.columns]
                df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
                
            except StopIteration:
                t_end = time()
                print(f'Table {table} created and all records inserted - %.2f seconds' % (t_end - t_start))
                break


In [3]:
if __name__ == '__main__':

    # Loading env variables
    load_dotenv(dotenv_path=Path('.\.env'))

    user = os.getenv('user')
    password = os.getenv('password')
    host = os.getenv('host')
    #port = os.getenv('port')
    database = os.getenv('database')
    print("database: ",database)
# Calling main function
create_in_mysql(user, password, host,database)

database:  ecommerce
Table dim_customers created and all records inserted - 0.21 seconds
Table dim_products created and all records inserted - 0.01 seconds
Table fact_sales created and all records inserted - 0.90 seconds
Table state_region_mapping created and all records inserted - 0.01 seconds


In [4]:
# Creating Postgre SQL connection, passing user and password as arguments when running script
con_string=f'mysql+pymysql://{user}:{password}@{host}/{database}'
engine = create_engine(con_string)

In [5]:
#tbl_list=  ['tbl_customers','tbl_products','tbl_fact_sales','tbl_state_region_mapping']
def get_query_string(table):
    query_string="select * from {}".format(table)
    query="""
            {}
            """.format(query_string)
    return query


In [6]:
df_customers = pd.read_sql(get_query_string('tbl_customers'),engine)

In [7]:
df_customers.head()

,customer_id,order_city,order_postal,order_state,latitude,longitude
0,18287,Rock Springs,82901-7321,WY,41.5869,-109.2048
1,18283,JACKSON,83001-9460,WY,43.4800,-110.7618
2,18282,Greybull,82426,WY,44.4892,-108.0562
3,18281,EVANSTON,82930-4706,WY,41.2680,-110.9633
4,18280,CHEYENNE,82001-7901,WY,41.1400,-104.8202


In [13]:
#saving this file
csv_name = "customers.csv"
df_customers.to_csv(csv_name, index=False)

In [8]:
df_products = pd.read_sql(get_query_string('tbl_products'),engine)

In [9]:
df_products.head()

,stock_code,weight,landed_cost,shipping_cost_1000_mile,description,category
0,22423,40.0,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food
1,85123A,2.0,6.0,5.0,Dog and Puppy Pads,Disposables
2,47566,6.0,18.0,12.0,"Memory Foam Pet Beds for Small, Medium, and La...",Disposables
3,85099B,2.0,5.0,4.0,Earth Rated Dog Poop Bags,Disposables
4,23084,2.0,6.0,6.0,Templation Soft Cat Treats,Pet Food


In [14]:
#saving this file
csv_name = "products.csv"
df_products.to_csv(csv_name, index=False)

In [19]:
df_fact_sales = pd.read_sql(get_query_string('tbl_fact_sales'),engine)

In [20]:
df_fact_sales.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365.0,1,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367.0,6,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1,25.50,25.50
3,12/1/2020 9:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371.0,14,214.20,15.30
4,12/1/2020 9:02,17850.0,Dog and Puppy Pads,85123A,536373.0,1,15.30,15.30


In [18]:
df_fact_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25065 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  25065 non-null  object 
 1   customer_id       20649 non-null  float64
 2   description       25065 non-null  object 
 3   stock_code        25065 non-null  object 
 4   invoice_no        24404 non-null  float64
 5   quantity          25065 non-null  int64  
 6   sales             25065 non-null  float64
 7   unit_price        25065 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.5+ MB


In [23]:
# as per the client, the records with blank invoice numbers should be filtered.
df_fact_sales_1= df_fact_sales[df_fact_sales['invoice_no'].notna()]

In [25]:
df_fact_sales_1.customer_id.fillna(df_fact_sales_1.invoice_no, inplace=True)

C:\Users\anita\AppData\Local\Temp\ipykernel_44524\1092005556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fact_sales_1.customer_id.fillna(df_fact_sales_1.invoice_no, inplace=True)


In [26]:
df_fact_sales_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24404 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  24404 non-null  object 
 1   customer_id       24404 non-null  float64
 2   description       24404 non-null  object 
 3   stock_code        24404 non-null  object 
 4   invoice_no        24404 non-null  float64
 5   quantity          24404 non-null  int64  
 6   sales             24404 non-null  float64
 7   unit_price        24404 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.7+ MB


In [27]:
df_fact_sales_1.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850.0,Dog and Puppy Pads,85123A,536365.0,1,15.30,15.30
1,12/1/2020 8:34,13047.0,Pet Odor Eliminator,84879,536367.0,6,60.84,10.14
2,12/1/2020 8:34,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1,25.50,25.50
3,12/1/2020 9:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371.0,14,214.20,15.30
4,12/1/2020 9:02,17850.0,Dog and Puppy Pads,85123A,536373.0,1,15.30,15.30


df['ID'] = df['ID'].astype(int).astype(str)
Or since there are NaN with:

df['ID'] = df['ID'].apply(lambda x: x if pd.isnull(x) else str(int(x)))

In [28]:
df_fact_sales_1['customer_id'] = df_fact_sales_1['customer_id'].apply(lambda x: x if pd.isnull(x) else str(int(x)))

C:\Users\anita\AppData\Local\Temp\ipykernel_44524\1004575832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fact_sales_1['customer_id'] = df_fact_sales_1['customer_id'].apply(lambda x: x if pd.isnull(x) else str(int(x)))


In [30]:
df_fact_sales_1['invoice_no'] = df_fact_sales_1['invoice_no'].apply(lambda x: x if pd.isnull(x) else str(int(x)))

C:\Users\anita\AppData\Local\Temp\ipykernel_44524\3171865490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fact_sales_1['invoice_no'] = df_fact_sales_1['invoice_no'].apply(lambda x: x if pd.isnull(x) else str(int(x)))


In [31]:
df_fact_sales_1.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,12/1/2020 8:26,17850,Dog and Puppy Pads,85123A,536365,1,15.30,15.30
1,12/1/2020 8:34,13047,Pet Odor Eliminator,84879,536367,6,60.84,10.14
2,12/1/2020 8:34,13047,Rechargeable Pet Nail Grinder,22960,536368,1,25.50,25.50
3,12/1/2020 9:00,13748,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371,14,214.20,15.30
4,12/1/2020 9:02,17850,Dog and Puppy Pads,85123A,536373,1,15.30,15.30


In [34]:
df_fact_sales_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24404 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_date  24404 non-null  object 
 1   customer_id       24404 non-null  object 
 2   description       24404 non-null  object 
 3   stock_code        24404 non-null  object 
 4   invoice_no        24404 non-null  object 
 5   quantity          24404 non-null  int64  
 6   sales             24404 non-null  float64
 7   unit_price        24404 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1.7+ MB


In [32]:
#saving this file
csv_name = "fact_sales_cleaned.csv"
df_fact_sales_1.to_csv(csv_name, index=False)

In [21]:
df_state_region_mapping = pd.read_sql(get_query_string('tbl_state_region_mapping'),engine)

In [22]:
#saving this file
csv_name = "state_region_mapping.csv"
df_state_region_mapping.to_csv(csv_name, index=False)